In [43]:
from torchvision import models
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image

In [44]:
# Load model dataset
resnet101 = torch.load("resnet101.pt")

In [45]:
print(resnet101)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

In [47]:
# ensure pick the right mods
resMod1_submodel_1 = list(resnet101.children())[:4]
resMod1_submodel_1 = nn.Sequential(*resMod1_submodel_1)

In [48]:
submodel_res_1 = list(resnet101.children())[4:5]
submodel_res_1 = nn.Sequential(*submodel_res_1)

In [49]:
for n in submodel_res_1.children():
  submodel_res_1_1 = nn.Sequential(n[:1])
  submodel_res_1_2 = nn.Sequential(n[1:2])
  submodel_res_1_3 = nn.Sequential(n[2:])

In [50]:
submodel_res_2 = list(resnet101.children())[5:6]
submodel_res_2 = nn.Sequential(*submodel_res_2)

In [51]:
for n in submodel_res_2.children():
  submodel_res_2_1 = nn.Sequential(n[:1])
  submodel_res_2_2 = nn.Sequential(n[1:2])
  submodel_res_2_3 = nn.Sequential(n[2:])

In [52]:
submodel_res_3 = list(resnet101.children())[6:7]
submodel_res_3 = nn.Sequential(*submodel_res_3)

submodel_res_4 = list(resnet101.children())[7:8]
submodel_res_4 = nn.Sequential(*submodel_res_4)

In [53]:
for n in submodel_res_3.children():
  submodel_res_3_1 = nn.Sequential(n[:8])
  submodel_res_3_2 = nn.Sequential(n[8:16])
  submodel_res_3_3 = nn.Sequential(n[16:])

print(submodel_res_3_3)

Sequential(
  (0): Sequential(
    (16): Bottleneck(
      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (17): Bottleneck(
      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runni

In [54]:
# extracts the adaptive avg pool mod
submodel_avgpool = list(resnet101.children())[8:9]
submodel_avgpool = nn.Sequential(*submodel_avgpool)

In [55]:
# creates a flatten layer to feed the output of the avg pool mod to the Linear mod
submodel_flatten = nn.Sequential(Flatten())

In [56]:
# This mod includes both the Avg Pool and Flatten sequentially
# Last 3 important modules
modules_2 = list(resnet101.children())[9:]
resMod2 = nn.Sequential(submodel_avgpool, submodel_flatten, *modules_2)
print(resMod2)

Sequential(
  (0): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (1): Sequential(
    (0): Flatten()
  )
  (2): Linear(in_features=2048, out_features=1000, bias=True)
)


In [57]:
# Prepare a transform to get the input image into a format (e.g., x,y dimensions) the classifier expects
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])

img = Image.open("input.jpg")

# Apply the transform to the image.
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

with open("classes.txt") as f:
    classes = [line.strip() for line in f.readlines()]

In [58]:
torch.save(resMod1_submodel_1, "resMod1_submodel_1.pt")
torch.save(submodel_res_1_1, "submodel_res_1_1.pt")
torch.save(submodel_res_1_2, "submodel_res_1_2.pt")
torch.save(submodel_res_1_3, "submodel_res_1_3.pt")
torch.save(submodel_res_2_1, "submodel_res_2_1.pt")
torch.save(submodel_res_2_2, "submodel_res_2_2.pt")
torch.save(submodel_res_2_3, "submodel_res_2_3.pt")
torch.save(submodel_res_3_1, "submodel_res_3_1.pt")
torch.save(submodel_res_3_2, "submodel_res_3_2.pt")
torch.save(submodel_res_3_3, "submodel_res_3_3.pt")
torch.save(submodel_res_4, "submodel_res_4.pt")
torch.save(resMod2, "resMod2.pt")


In [59]:
resMod1_submodel_1.eval()
submodel_res_1_1.eval()
submodel_res_1_2.eval()
submodel_res_1_3.eval()
submodel_res_2_1.eval()
submodel_res_2_2.eval()
submodel_res_2_3.eval()
submodel_res_3_1.eval()
submodel_res_3_2.eval()
submodel_res_3_3.eval()
submodel_res_4.eval()
resMod2.eval()
# resnet101.eval()

Sequential(
  (0): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (1): Sequential(
    (0): Flatten()
  )
  (2): Linear(in_features=2048, out_features=1000, bias=True)
)

In [60]:
output_resMod1_submodel_1 = resMod1_submodel_1(batch_t)
output_submodel_res_1_1 = submodel_res_1_1(output_resMod1_submodel_1)
output_submodel_res_1_2 = submodel_res_1_2(output_submodel_res_1_1)
output_submodel_res_1_3 = submodel_res_1_3(output_submodel_res_1_2)
output_submodel_res_2_1 = submodel_res_2_1(output_submodel_res_1_3)
output_submodel_res_2_2 = submodel_res_2_2(output_submodel_res_2_1)
output_submodel_res_2_3 = submodel_res_2_3(output_submodel_res_2_2)
output_submodel_res_3_1 = submodel_res_3_1(output_submodel_res_2_3)
output_submodel_res_3_2 = submodel_res_3_2(output_submodel_res_3_1)
output_submodel_res_3_3 = submodel_res_3_3(output_submodel_res_3_2)
output_submodel_res_4 = submodel_res_4(output_submodel_res_3_3)
final_output = resMod2(output_submodel_res_4)

In [61]:
# Sort the predictions.
_, indices = torch.sort(final_output, descending=True)

# Convert into percentages.
percentage = torch.nn.functional.softmax(final_output, dim=1)[0] * 100

# Print the 5 most likely predictions.
with open("result_Resnet101.txt", "w") as outfile:
    outfile.write(str([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]))

print(open('result_Resnet101.txt', 'r').read()) 

[('Labrador retriever', 48.255558013916016), ('dingo, warrigal, warragal, Canis dingo', 7.900781154632568), ('golden retriever', 6.91691780090332), ('Eskimo dog, husky', 3.6434459686279297), ('bull mastiff', 3.046128273010254)]
